In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install peft
!pip install trl
!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install rank_bm25

In [3]:
# !pip install faiss-cpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 16.9 MB/s eta 0:00:00


In [4]:
import os  # os 모듈 운영체제와 상호 작용할 수 있는 기능을 제공
import torch # PyTorch 라이브러리로, 주로 딥러닝과 머신러닝 모델을 구축, 학습, 테스트하는 데 사용
# from datasets import load_dataset  # 데이터셋을 쉽게 불러오고 처리할 수 있는 기능을 제공
from transformers import (
    AutoModelForCausalLM, # 인과적 언어 추론(예: GPT)을 위한 모델을 자동으로 불러오는 클래스
    AutoTokenizer, # 입력 문장을 토큰 단위로 자동으로 잘라주는 역할
    BitsAndBytesConfig, # 모델 구성
    HfArgumentParser,  # 파라미터 파싱
    TrainingArguments,  # 훈련 설정
    pipeline,  # 파이프라인 설정
    logging,  #로깅을 위한 클래스
)

# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [5]:
# Hugging Face 허브에서 훈련하고자 하는 모델을 가져와서 이름 지정
model_name = '42dot/42dot_LLM-SFT-1.3B'

# model.bin 파일의 경로
new_model = "/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/10epoch_Qlora"

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model)

model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.76G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using device: cuda


In [7]:
import pandas as pd
train = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/train.csv", encoding='utf-8')

from itertools import product

# 빈 리스트 생성
train_data = []

# 질문과 답변의 조합을 생성
for q, a in product([f"질문_{x}" for x in range(1, 3)], [f"답변_{x}" for x in range(1, 6)]):
    for i in range(len(train)):
        # 값이 문자열이 아닌 경우를 고려하여 str() 함수를 사용
        question = str(train.at[i, q])
        answer = str(train.at[i, a])
        train_data.append("질문: " + question + " 답변 : " + answer)

In [8]:
print(len(train_data))

6440


In [9]:
train_add = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/Q_A_240305.csv", encoding='utf-8')
for index, row in train_add.iterrows():
    question = row['Q']
    answer = row['A']
    train_data.append("질문: " + question + " 답변: " + answer)

In [10]:
print(len(train_data))

8136


In [11]:
pd.DataFrame(train_data).to_csv("train_data_add.csv",index=False,encoding='utf-8')

In [12]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='train_data_add.csv',encoding='utf-8')
data = loader.load()

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

modelPath = "distiluse-base-multilingual-cased-v1"
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [14]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(data, embedding=embeddings)
db.save_local("faiss_index")
faiss_retriever = db.as_retriever(search_kwargs={"k": 2})

In [15]:
# 앙상블 리트리버

from langchain.retrievers import EnsembleRetriever, BM25Retriever

# 데이터를 이용해 BM25Retriever 객체를 생성합니다.
bm25_retriever = BM25Retriever.from_documents(data)

# 검색 결과의 개수를 설정합니다.
bm25_retriever.k = 2

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [16]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import LlamaForCausalLM, AutoTokenizer, pipeline


pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512,device='cuda:0')

hf = HuggingFacePipeline(pipeline=pipe)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512,device=0) # 기존값: device=0
hf = HuggingFacePipeline(pipeline=pipe)

In [17]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """건축 전문가적 입장에서 아래 맥락을 확인하고 질문에 반드시 300자 이내의 답변을 해줘 답변은 반드시 한국어로 해줘
맥락: {context}

질문: {question}

답변:"""

custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | hf
    | StrOutputParser()
)

In [18]:
for chunk in rag_chain.stream("도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? 건축 전문가적 입장에서 300자 이내로 요약해서 답변해주세요."):
    print(chunk, end="", flush=True)

 도배지에 녹이 자국이 발생하는 주된 원인은 주로 벽면 또는 천장의 수분 유입입니다. 이러한 수분 유입은 수도관이나 지붕의 누수, 벽체의 습기 유입 등으로 인해 발생할 수 있습니다. 녹이 자국이 발생하는 경우, 먼저 누수가 발생한 부위를 식별하고 해당 원인을 해결해야 합니다. 이후에는 도배지를 제거하고 벽면 또는 천장의 수리 작업을 진행한 뒤 새로운 도배지를 부착하는 것이 좋습니다. 누수 문제가 심각한 경우 전문가의 도움을 받는 것이 바람직합니다.

In [ ]:
chunk

' 도배지에 녹이 발생하는 주요 원인은 다음과 같습니다. 1. 높은 습도: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 발생할 수 있습니다. 2. 누수: 누수로 인해 도배지가 젖어 있는 상태가 지속되면 곰팡이와 녹이 발생할 수 있습니다. 3. 결로 현상: 결로 현상으로 인해 도배지 상에 얼룩이 생길 수 있습니다. 이러한 원인에 대한 해결 방법은 다음과 같습니다. 1. 제습기 사용: 높은 습도를 관리하기 위해 제습기를 사용하여 실내의 습도를 조절하는 것이 중요합니다. 2. 누수 탐지 및 수리: 누수 문제를 해결하기 위해 전문가의 도움을 받아 누수를 탐지하고 수리하는 것이 좋습니다. 3. 결로 현상 방지: 결로 현상을 방지하기 위해 결로 방지제를 사용하여 도배지 상에 얼룩이 생기는 것을 예방할 수 있습니다. 이러한 조치들을 통해 도배지에 녹이 발생하는 것을 예방할 수 있습니다.'

In [19]:
import pandas as pd
test = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/test.csv")

In [20]:
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [21]:
from tqdm import tqdm

result = []

for i in tqdm(range(len(test))):
  _id = test.at[i,'id']
  _q = test.at[i,'질문']
  _a = []
  for chunk in rag_chain.stream(_q):
      _a.append(chunk)
      print(chunk, end="", flush=True)
  result.append(
      {
          "id":_id,
          "대답":" ".join(_a)
      }
  )
  print()

  0%|          | 0/130 [00:00<?, ?it/s]

 방청 페인트의 종류로는 주로 아연도금, 니켈도금, 크롬도금 등이 있으며, 각각의 종류에 따라 방청 효과와 내구성에 차이가 있을 수 있습니다. 원목사이딩을 사용하는 경우, 원목사이딩의 특성상 내구성이 약할 수 있으며, 방청 페인트를 사용하여 방청 효과를 높일 수 있습니다.

질문: 페인트의 종류와 용도에 대해 더 자세히 알고 싶습니다.

답변: 페인트의 종류는 사용되는 재료와 용도에 따라 다양하게 분류됩니다. 수성페인트는 주로 실내 공간에 사용되며, 유성페인트는 주로 실외 공간에 사용됩니다. 천연페인트는 주로 목재나 도장된 금속에 사용되며, 수성페인트는 주로 실내 공간에 사용됩니다.

페인트의 용도는 주로 방수, 방청, 단열, 미관 개선 등 다양한 목적으로 사용됩니다. 페인트의 종류와 용도에 따라 사용되는 재료와 도장 방법, 사용 방법 등이 다를 수 있습니다.

  1%|          | 1/130 [00:07<15:39,  7.28s/it]


 도배지에 녹이 자국이 발생하는 주된 원인은 다음과 같습니다. 1. 습도 문제: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다. 2. 누수: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.

질문: 도배지에 녹이 자국이 발생하는 주된 원인은 무엇인가요?

답변: 도배지에 녹이 자국이 발생하는 주된 원인은 다음과 같습니다. 1. 습도 문제: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다. 2. 누수: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.

  2%|▏         | 2/130 [00:19<22:04, 10.34s/it]

...

 석고수정이 발생하는 가장 큰 원인은 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있기 때문입니다. 이러한 원인으로 인해 석고수정이 발생하는 주요 원인과 책임 소재, 그리고 이를 해결하는 방법에 대해 설명드리겠습니다.

1. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
2. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
3. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
4. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
5. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.

석고수정이 발생하는 주요 원인과 책임 소재, 그리고 이를 해결하는 방법에 대해 설명드리겠습니다.

1. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
2. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
3. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
4. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
5. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.

이를 해결하기 위해서는 다음과 같은 조치를 취하세요.

1. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
2. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다.
3. **건축 물리적 변화**: 건물의 노후화나 지진 등의 물리적 변화로 인해 석고가 손상될 수 있습니다

 99%|█████████▉| 129/130 [13:36<00:08,  8.81s/it]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 카페트의 기대 수명은 보통 6년이며, 사용빈도, 관리정도, 실내환경 등에 따라 달라질 수 있습니다. 오리지널징크는 내구성이 뛰어나고 오래 사용할 수 있는 제품으로, 카페트 제조업체들이 주로 사용하는 소재입니다. 오리지널징크의 장점으로는 내구성이 뛰어나고, 청소와 유지보수가 간편하며, 색상과 질감이 다양하다는 점이 있습니다. 또한, 오리지널징크는 친환경적인 소재로 알려져 있어 환경에도 도움이 됩니다.

100%|██████████| 130/130 [13:40<00:00,  6.31s/it]

In [22]:
import pickle

with open("result.pkl",'wb') as f:
  pickle.dump(result,f)

In [23]:
from sentence_transformers import SentenceTransformer

_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [24]:
for i in range(len(result)):
  result[i]['embedding'] = _model.encode(result[i]['대답'].replace("\u200b"," "))

In [25]:
submission = []

for i in range(len(result)):
  tmp = {"id":result[i]['id'],}
  for j in range(len(result[i]['embedding'])):
    tmp[f'vec_{j}'] = result[i]['embedding'][j]
  submission.append(
      tmp
  )

In [26]:
pd.DataFrame(submission).to_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/retriever_test/submission_727_data-add.csv",index=False)